In [18]:
#import necessary libraries
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import scipy.ndimage as nd
from tensorflow import keras
%matplotlib inline
import pylab as pl
from IPython import display

import tensorflow as tf
import numpy as np
import sys
sys.path.append("..") # Adds higher directory to python modules path.




import h5py
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Concatenate
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D

# from tensorflow.examples.tutorials.mnist import input_data

In [19]:
import branchingdnn as branching
from branchingdnn.utils import * 
from branchingdnn.profiler import model_profiler as profiler
from branchingdnn.dataset import prepare
from branchingdnn.branches import branch
from branchingdnn.initNeptune import Neptune

In [3]:
 dataset = prepare.dataset_distil(tf.keras.datasets.cifar10.load_data(),32,5000,22500,(227,227))

trainSize 45000
testSize 10000
<BatchDataset shapes: (((32, 227, 227, 3), (32, 1)), (32, 1)), types: ((tf.float32, tf.uint8), tf.uint8)>
<BatchDataset shapes: (((32, 227, 227, 3), (32, 1)), (32, 1)), types: ((tf.float32, tf.uint8), tf.uint8)>


In [20]:
modelName = "alexNetv6.hdf5"
saveName = "alexNetv6_distil_new"
transfer = True
customOptions="CrossE"
numpEpocs = 10

x = tf.keras.models.load_model('../models/{}'.format(modelName))
# x.summary()
if saveName =="":
    saveName = modelName
# tf.keras.utils.plot_model(x, to_file="{}.png".format(saveName), show_shapes=True, show_layer_names=True)
# funcModel = models.Model([input_layer], [prev_layer])
# funcModel = branch.add(x,["dense","conv2d","max_pooling2d","batch_normalization","dense","dropout"],branch.newBranch)
# dataset = prepare.dataset_distil(tf.keras.datasets.cifar10.load_data(),32,5000,22500,(227,227))


In [21]:
class Notebook_EvidenceEndpoint(keras.layers.Layer):
        def __init__(self, name=None, **kwargs):
            super(Notebook_EvidenceEndpoint, self).__init__(name=name)
            self.num_outputs = 10
            self.loss_coefficient = 1
            self.feature_loss_coefficient = 1
            self.kl = tf.keras.losses.KLDivergence()
            self.global_step = tf.Variable(initial_value=0.0, name='global_step', trainable=False)
            self.crossE = tf.keras.losses.SparseCategoricalCrossentropy()
            self.loss_fn = utils.evidence_loss(global_step=self.global_step)
            self.temperature = 10
            self.alpha = .1
            self.lmb = 0.005
        def build(self, input_shape):
            self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]),
                                                    self.num_outputs])

        def call(self, inputs, labels):
            outputs = tf.matmul(inputs,self.kernel)
            # Compute the training-time loss value and add it
            # to the layer using `self.add_loss()`.
            #loss functions are (True, Prediction)
            # softmax = tf.nn.softmax(inputs)
            # print("inputs",input)
            #loss 1. normal loss, predictions vs labels
            evidence, normal_loss = self.loss_fn(labels, outputs)
            # print(self.weights, self.lmb)
            # l2_loss = tf.nn.l2_loss(self.weights)# * self.lmb
            # print("l2_loss",l2_loss)
            # print(evidence.shape)
            # print(normal_loss)
            total_loss =tf.reduce_mean(normal_loss)# + l2_loss
            # print(total_loss)
            total_evidence = tf.reduce_sum(evidence,1, keepdims=True) 
            # print(total_evidence)
            pred = tf.argmax(outputs, 1)
            # truth = tf.argmax(labels, 1)
            # print("labels",labels)
            truth = tf.cast(labels,tf.int64)
            # print("evid", evidence)
            # print("loss", normal_loss)
            # print("truth",truth)
            # print("pred",pred)
            match = (tf.equal(tf.reshape(truth,(1,32)),pred))
            # match = tf.where(tf.math.equal(pred, tf.cast(tf.reshape(truth,(32,1)),'int64')))
            match = tf.cast(match,tf.float32)
            # match = tf.reshape(tf.cast(tf.equal(pred, truth), tf.float32),(-1,1))
            # print("match",match)
            mean_avg = tf.reduce_mean(total_evidence)
            # print("mean_Avg")
            # print("match",match)
            mean_succ = tf.reduce_sum(tf.reduce_sum(evidence,1, keepdims=True)* match) / tf.reduce_sum(match+1e-20)
            # print("mean_fail")
            # side_x = tf.reduce_sum(tf.reduce_sum(evidence,1, keepdims=True)*(1-match))
            # print("side_x",side_x)
            # side_y = (tf.reduce_sum(tf.abs(1-match))+1e-20)
            # print("side_y",side_y)
            # div = tf.reduce_sum(side_x/side_y)
            # print("div",div)
            mean_fail = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(evidence,1, keepdims=True)*(1-match)) / (tf.reduce_sum(tf.abs(1-match))+1e-20) )
            # print('end')
            # self.add_metric(normal_loss, name=self.name+"_normal_loss")
            # self.add_metric(evidence, name=self.name+"_evidence")
            # self.add_metric(mean_avg, name=self.name+"_mean_ev_avg")
            # self.add_metric(mean_succ, name=self.name+"_mean_ev_succ")
            # self.add_metric(mean_fail, name=self.name+"_mean_ev_fail")
            # print('metrics')
            # self.add_loss(total_loss)
            softmax = tf.nn.softmax(outputs)
            CrossE_loss= self.crossE(labels,softmax)
            self.add_loss(CrossE_loss)

            # print("pred",inputs)
            # print("loss")
            #NOTE
            # The total loss is different from parts_loss because it includes the regularization term.
            # In other words, loss is computed as loss = parts_loss + k*R, where R is the regularization term 
            # (typically the L1 or L2 norm of the model's weights) and k a hyperparameter that controls the 
            # contribution of the regularization loss in the total loss.
            return outputs

In [22]:
def notebook_Branch_bottleneck(prevLayer, targets=None, teacher_sm=None, teaching_features=None):
        print("targets::::",targets)
        print("teacher_sm::::",teacher_sm)
        print("teaching_features::::",teaching_features)
        if prevLayer.shape[1] == 4096:
            #don't add a feature distil to the last branch
            teaching_features = None
        if teaching_features is not None:
            branchLayer = branch.bottleneck(prevLayer,teaching_features)
            branchLayer = branch.FeatureDistillation(name=tf.compat.v1.get_default_graph().unique_name("branch_teaching"))(branchLayer,teaching_features)    
            branchLayer = layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(branchLayer)
        else:
            # branchLayer = branch.bottleneck2(prevLayer)
            print("no teaching feature Provided, bottleneck and teaching loss skipped")
            branchLayer = layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(prevLayer)

        branchLayer = layers.Dense(124, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch124"))(branchLayer)
        branchLayer = layers.Dense(64, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch64"))(branchLayer)
        # branchLayer = layers.Dense(10, name=tf.compat.v1.get_default_graph().unique_name("branch_output"))(branchLayer)
        # if targets is None and teacher_sm is None:
            # output = (layers.Softmax(name=tf.compat.v1.get_default_graph().unique_name("branch_softmax"))(branchLayer))
        # else:
            # output = branch.BranchEndpoint(name=tf.compat.v1.get_default_graph().unique_name("branch_softmax"))(branchLayer, targets, teacher_sm)
        # output = (layers.Softmax(name=tf.compat.v1.get_default_graph().unique_name("branch_softmax"))(branchLayer))
        output = Notebook_EvidenceEndpoint(name=tf.compat.v1.get_default_graph().unique_name("branch_output"))(branchLayer, targets)

        return output

In [23]:
funcModel = branch.add_distil(x,["max_pooling2d"],[notebook_Branch_bottleneck],exact=True)
#so to self distil, I have to pipe the loss from the main exit back to the branches.
# funcModel.summary()
funcModel.save("models/{}".format(saveName))


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 55, 55, 96)   34944       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 55, 55, 96)   384         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 27, 27, 96)   0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [24]:
model = funcModel

epocs = 10
save = False
transfer = True
saveName = saveName
customOptions="CrossE"
tags =["v6","drt"]

logs = []
num_outputs = len(model.outputs) # the number of output layers for the purpose of providing labels
train_ds, test_ds, validation_ds = dataset
# train_ds, test_ds, validation_ds = prepare.prepareMnistDataset(dataset, batch_size=32)

#Freeze main branch layers
#how to iterate through layers and find main branch ones?
#simple fix for now: all branch nodes get branch in name.
if transfer: 
    for i in range(len(model.layers)):
        # print(model.layers[i].name)
        if "branch" in model.layers[i].name:
            # print("setting branch layer training to true")
            model.layers[i].trainable = True
        else: 
            # print("setting main layer training to false")
            model.layers[i].trainable = False               
else:
    for i in range(len(model.layers)):
        # print(model.layers[i].name)
        model.layers[i].trainable = True
        # print("setting layer training to True")
        # model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=keras.optimizers.Adam(),metrics=["accuracy"])


if customOptions == "CrossE": 
    print("customOption: CrossE")
    model.compile(loss='SparseCategoricalCrossentropy',  optimizer=tf.optimizers.SGD(lr=0.01,momentum=0.9), metrics=['accuracy'],run_eagerly=True)
elif customOptions == "CrossE_Eadd":
    print("customOption: CrossE_Eadd")
    entropyAdd = entropyAddition_loss()
    model.compile(optimizer=tf.optimizers.SGD(lr=0.01,momentum=0.9,clipvalue=0.5), loss=[keras.losses.SparseCategoricalCrossentropy(),entropyAdd,entropyAdd,entropyAdd], metrics=['accuracy',confidenceScore, unconfidence],run_eagerly=True)
    # model.compile(optimizer=tf.optimizers.SGD(lr=0.001), loss=[crossE_test, entropyAdd, entropyAdd, entropyAdd], metrics=['accuracy',confidenceScore, unconfidence],run_eagerly=True)
else:
    print("customOption: Other")
# model.compile(loss=entropyAddition, optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'],run_eagerly=True)
    model.compile(loss='SparseCategoricalCrossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy',confidenceDifference],run_eagerly=True)

run_logdir = get_run_logdir(model.name)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
# print("after reset:")
# test_scores = model.evaluate(test_ds, verbose=2)
# print("finish eval")
# printTestScores(test_scores,num_outputs)

if saveName =="":
    newModelName = "{}_branched.hdf5".format(model.name )
else:
    newModelName = saveName
checkpoint = keras.callbacks.ModelCheckpoint("models/{}.hdf5".format(newModelName), monitor='val_acc', verbose=1, mode='max')

# neptune_cbk = Neptune.getcallback(name = newModelName, tags =tags)
# print("epoc: {}".format(j))
# results = [j]           
# history =model.fit(train_ds,
#         epochs=epocs,
#         validation_data=validation_ds,
#         validation_freq=1,
#         # batch_size=1,
#         callbacks=[tensorboard_cb,checkpoint])
#                     # callbacks=[tensorboard_cb,checkpoint])
# print(history)
# test_scores = model.evaluate(test_ds, verbose=2)
# print("overall loss: {}".format(test_scores[0]))
# if num_outputs > 1:
#     for i in range(num_outputs):
#         print("Output {}: Test loss: {}, Test accuracy {}".format(i, test_scores[i+1], test_scores[i+1+num_outputs]))
#         results.append("Output {}: Test loss: {}, Test accuracy {}".format(i, test_scores[i+1], test_scores[i+1+num_outputs]))
# else:
#     print("Test loss:", test_scores[0])
#     print("Test accuracy:", test_scores[1])
# logs.append(results)

customOption: CrossE


In [25]:
model.fit(train_ds,
        epochs=epocs,
        validation_data=validation_ds,
        validation_freq=1,
        # batch_size=1,
        verbose=2)
                    # callbacks=[tensorboard_cb,checkpoint])


Epoch 1/10


In [ ]:
# print(history)
test_scores = model.evaluate(test_ds, verbose=2)
print("overall loss: {}".format(test_scores[0]))